# Setup

In [2]:
import os
import requests 
import pandas as pd 
import time
from google.colab import files

In [3]:
os.environ['TOKEN'] = # INSERT TOKEN HERE
env_label = # INSERT ENVIRONMENT LABEL HERE

In [6]:
# Header for autentication
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [5]:
headers = create_headers(os.environ['TOKEN'])

# Download data


In [7]:
def create_url(keyword, start_date, end_date, env_label, endpoint="30day"):
    # endpoint can be "30day" or "fullarchive"
    
    search_url = "https://api.twitter.com/1.1/tweets/search/{}.json".format(endpoint+"/"+env_label) 
    # change params based on the endpoint you are using
    query_params = {'query': keyword, 'fromDate': start_date, 'toDate': end_date}
    return (search_url, query_params)


# Continue the request for data until there's no relevant data wrt to the query_params
def connect_to_endpoint(url, headers, params, next_token = None):
    if next_token is not None and next_token != '':
      params['next'] = next_token
    response = requests.request("GET", url, headers = headers, params = params)
    if response.status_code != 200:  # there's an error (e.g. 404 not found)
        raise Exception(response.status_code, response.text)
    return response.json()


def get_data(keyword, start_time, end_time, next_token, env_label, endpoint):
  results = []
  while next_token is not None:
    url = create_url(keyword, start_time,end_time, env_label, endpoint)
    json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
    
    if "results" in json_response:
      results.extend(json_response["results"])

    if "next" in json_response:
        next_token = json_response["next"]
    else:
      next_token = None
    time.sleep(2)   # there's a limit on the requests per minutes
                    # so we wait 2 seconds before performing the next request
  return results


def get_single_response(keyword, start_time, end_time, env_label, endpoint):
  results = []
  url = create_url(keyword, start_time,end_time, env_label, endpoint)
  json_response = connect_to_endpoint(url[0], headers, url[1])
  
  if "results" in json_response:
    results.extend(json_response["results"])
  
  return results

In [8]:
# If you are using the 30day endpoint, make sure you specify dates that are within 30day range.
# Date format: Year-Month-Day Time

# WUHAN OUTBREAK AND EARLY PANDEMIC
start_date = "202001220000" 
end_date = "202002120000" 
endpoint = "fullarchive"

# KEYWORDS: Wuhan, China, Coronavirus

# ((((((2019-nCoV gives 0 results))))))

In [ ]:
# EARLY PANDEMIC - COVID
start_date = "202002120000" 
end_date = "202002200000" 
endpoint = "fullarchive"

# KEYWORDS: Wuhan, China, Coronavirus, Covid

In [ ]:
# AFTERWARD PHASE - DELTA
# delta began to spread in India in late 2020
# delta is the worldwide dominant variant in June/July 2021 
start_date = "?"
end_date = "?" 
endpoint = "fullarchive"

# KEYWORDS: Delta

In [ ]:
# AFTERWARD PHASE - OMICRON
# first case on November 22, 2021
start_date = None
end_date = None 
endpoint = "30day"

# KEYWORDS: Omicron

In [ ]:
# International News Accounts

# OUTSIDE CHINA
# from:BBCWorld, CNN, AP, AJEnglish, Reuters

# INSIDE CHINA
# from: XHNews, CGTNOfficial

# LANGUAGE
# lang:en

In [111]:
# DON'T RUN UNLESS YOU ARE SURE, WARNING !#!##@!!!
account = "AJEnglish"
keyword = "covid"
start_date = "202001220000" 
end_date = "202002220000" 
full_keyword = "from:"+account+" "+keyword+" lang:en"
all_tweets = get_data(full_keyword, start_date, end_date, '', env_label, endpoint)

In [112]:
all_tweets_df = pd.DataFrame(all_tweets)

In [113]:
len(all_tweets_df)

4

In [114]:
label = account+"_"+keyword+"_"+start_date+"_"+end_date
all_tweets_df.to_csv(label+".csv", index=False)
all_tweets_df.to_pickle(label+".pkl")
files.download(label+".csv")
files.download(label+".pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>